In [109]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [110]:
import anvil.server
anvil.server.connect('server_57AQBXHXBXQB3SV63LDEGSMA-UP4OJMY33G3H35TG')

In [111]:
import joblib
import pickle
import json
import requests
import gzip
import numpy as np
import pandas as pd

In [112]:
knn_from_joblib = joblib.load('model.pkl')

/usr/local/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator Lasso from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [118]:
def ou(o):
      return o/1e+4

@anvil.server.callable
def output(instagram, twitter, discord, opensea, volumeCount):
  s = [instagram, twitter, discord, opensea, volumeCount]
  if(sum(s)<100): 
    return 0.1

  for i in range(len(s)):
    s[i]=ou(s[i])

  po=knn_from_joblib.predict(np.array(s).reshape(1, -1))
  ans=po*(19.25)
  final_output = ans[0]
  return np.abs(final_output)

In [119]:
output(163528.0	,36954.0	,7675.0	,3100	,1600.0)

0.5528412413729415

In [120]:
# Only instagram and volumecount increases floor price. 
# Does not make sense to tell user to sell more nfts(increasing volume count) so they could raise their floor price. Will only be working with instagram followers.
coeffs = knn_from_joblib.coef_
coeffs

array([-4.12891932e-05, -1.15745661e-03, -1.83947076e-03, -7.72601535e-02,
        5.09552352e-02])

In [121]:
# Use model to spit out price
# If we want a 20% increase -> [predicted price (from model)] * [0.2] = [increase]
# [increase] / [0.00501408(instagram coef)] = [number of instagram followers you have to increase to achieve desired floor price]

In [122]:
@anvil.server.callable
def rec_insta(predicted_price, desired_price):
  coeff = np.abs(coeffs[0])
  increase = desired_price - predicted_price
  num_followers = round(increase / coeff)
  return num_followers

In [123]:
rec_insta(7, 13)

145316

In [124]:
@anvil.server.callable
def rec_twitter(predicted_price, desired_price): 
  coeff = np.abs(coeffs[1])
  increase = desired_price - predicted_price
  num_followers = round(increase / coeff)
  return num_followers 

In [125]:
@anvil.server.callable
def rec_discord(predicted_price, desired_price): 
  coeff = np.abs(coeffs[2])
  increase = desired_price - predicted_price
  num_followers = round(increase / coeff)
  return num_followers 

In [126]:
@anvil.server.callable
def rec_opensea(predicted_price, desired_price): 
  coeff = np.abs(coeffs[3])
  increase = desired_price - predicted_price
  num_followers = round(increase / coeff)
  return num_followers 